In [1]:
%run imports_and_functions.ipynb

from dbfread import DBF

C:\Users\Jason\AppData\Local\conda\conda\envs\DSI-6\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [7]:
# income data files come in .dbf, must install this to read from python

# !pip install dbfread

### Creating Final Businesses Dataframe within City Radius

In [2]:
df = pd.read_csv('MiamiBusinesses.csv', sep=';', compression='gzip')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10571 entries, 0 to 10570
Data columns (total 12 columns):
Unnamed: 0       10571 non-null int64
alias            10571 non-null object
categories       10571 non-null object
coordinates      10571 non-null object
distance         10571 non-null float64
latitude         10531 non-null float64
longitude        10531 non-null float64
main_category    10571 non-null object
name             10571 non-null object
price            4185 non-null object
rating           10571 non-null float64
review_count     10571 non-null float64
dtypes: float64(5), int64(1), object(6)
memory usage: 991.1+ KB


In [3]:
def dropduplicate_yelp_entries(df) :
    dups = df[df[['latitude','longitude','alias']].duplicated(keep=False)].sort_values('latitude')
    mask = dups[['latitude','longitude','alias']].duplicated(keep='first')
#     print(dups)
    for index in dups[mask].index :
        dupset = dups[(dups['alias']==dups.loc[index,'alias']) & (dups['longitude']==dups.loc[index,'longitude'])].sort_values(
            'review_count', ascending=False)
        dropindex = dupset.iloc[1:,0].index
        df.drop(dropindex, axis=0, inplace=True)

def getPriceRange(string) :
    regex = re.compile("RestaurantsPriceRange2\\': \\'(\d)")
    try :
        match = regex.search(string)
        return int(match.groups(0)[0])
    except : return 0

def get_main_category(catstring) :
    string = ''
    try :
        for cat in main_categories :
            if cat in catstring : string = string + cat + ', '
    except : pass
    return string[0:(len(string)-2)]

def convertpricecolumn(pricesymbol) :
    try : num = len(pricesymbol)
    except : num = 0
    return num

In [4]:
print(df.shape)
dropduplicate_yelp_entries(df)
df.shape

(10571, 12)


(10568, 12)

In [5]:
# Still quite low percentage that have price field, but in new yelp data it's no longer found in
# attribute field under 'PriceRange2'
# df['price'] = df['attributes'].map(getPriceRange)
df['price'] = df['price'].map(convertpricecolumn)
df['price'].value_counts()

0    6385
2    2357
1    1201
3     478
4     147
Name: price, dtype: int64

In [6]:
# SET FOR EACH CITY: maxradius, testlat, testlong
maxradius = 6000
testlat = 25.786686
testlong = -80.237440 # Miami, FL

In [7]:
df['distance2'] = calcDistances(testlat, testlong, df)

df[df['distance2']<maxradius].shape

(8431, 13)

In [8]:
# yelp calculated distances seem within 30 meters of python geo-module calculations
# need to use these ones because of 'extra_squares' search where yelp returns distance to these
# new auxilary centers, not the true city center
df['distance'] = df['distance2']

In [33]:
# df[df[['latitude','longitude']].duplicated(keep=False)].sort_values('latitude')

In [9]:
rowstodrop = df[(df['distance']>maxradius) | df['latitude'].isnull() | df['longitude'].isnull()].index

df.drop(rowstodrop, axis=0, inplace=True)
df.shape

(8431, 13)

In [10]:
df['lat_index'] = round(df['latitude'], 3)
df['long_index'] = round(df['longitude'], 4)

df = df.set_index(['lat_index','long_index'], drop=False).sort_index()
df = df.sort_index(level = [0,1])

In [11]:
df.tail()

Unnamed: 0  \
lat_index long_index               
25.838    -80.2510         13023   
          -80.2411         13002   
          -80.2362          5152   
          -80.2254         13016   
25.839    -80.2234         10706   

                                                                  alias  \
lat_index long_index                                                      
25.838    -80.2510                                         miatex-miami   
          -80.2411    better-way-of-miami-house-of-bargains-thrift-s...   
          -80.2362                         safe-home-movers-west-park-2   
          -80.2254              elegant-beauty-supplies-and-salon-miami   
25.839    -80.2234                      miracle-fry-conchfritters-miami   

                                                             categories  \
lat_index long_index                                                      
25.838    -80.2510    [{'alias': 'fabricstores', 'title': 'Fabric St...   
          -80.2411    [{'alias': 'thrift_stores', 'title': 'Thrift S...   
          -80.2362    [{'alias': 'movers', 'title': 'Movers'}, {'ali...   
          -80.2254    [{'alias': 'cosmetics', 'title': 'Cosmetics & ...   
25.839    -80.2234    [{'alias': 'tradamerican', 'title': 'American ...   

                                                            coordinates  \
lat_index long_index                                                      
25.838    -80.2510     {'latitude': 25.838121, 'longitude': -80.251021}   
          -80.2411       {'latitude': 25.83847, 'longitude': -80.24113}   
          -80.2362    {'latitude': 25.8377510773947, 'longitude': -8...   
          -80.2254     {'latitude': 25.838165, 'longitude': -80.225418}   
25.839    -80.2234     {'latitude': 25.8390099, 'longitude': -80.22342}   

                         distance   latitude  longitude main_category  \
lat_index long_index                                                    
25.838    -80.2510    5858.711181  25.838121 -80.251021      shopping   
          -80.2411    5748.817455  25.838470 -80.241130      shopping   
          -80.2362    5658.723529  25.837751 -80.236153  homeservices   
          -80.2254    5829.126358  25.838165 -80.225418      shopping   
25.839    -80.2234    5964.755736  25.839010 -80.223420   restaurants   

                                                                   name  \
lat_index long_index                                                      
25.838    -80.2510                                               Miatex   
          -80.2411    Better Way of Miami House of Bargains Thrift S...   
          -80.2362                                     Safe Home Movers   
          -80.2254                      Elegant Beauty Supplies & Salon   
25.839    -80.2234                            Miracle Fry Conchfritters   

                      price  rating  review_count    distance2  lat_index  \
lat_index long_index                                                        
25.838    -80.2510        0     5.0           1.0  5858.711181     25.838   
          -80.2411        0     5.0           1.0  5748.817455     25.838   
          -80.2362        0     1.5          13.0  5658.723529     25.838   
          -80.2254        2     3.5           3.0  5829.126358     25.838   
25.839    -80.2234        1     4.5           6.0  5964.755736     25.839   

                      long_index  
lat_index long_index              
25.838    -80.2510      -80.2510  
          -80.2411      -80.2411  
          -80.2362      -80.2362  
          -80.2254      -80.2254  
25.839    -80.2234      -80.2234

In [12]:
cat_alias_dict = json.load(open('cat_alias_dict.json','r'))

def categories_to_alias2(dfrow) :
    catstring = dfrow['categories']
    if type(catstring) != str : return str(dfrow['price']) + '_other'
    rawlis = list(set(catstring.split(', ')))
    lis = []
    for item in rawlis :
        if 'alias' in item :
            alias = item.split(':')[1].strip().strip("'")
            lis += [alias]
#     print(lis)
    lis = list(map(get_alias2,lis))
    lis = [item for item in lis if item != '_other']
    lis += ['_' + dfrow['main_category']]
    # some new categories are not in category dictionary, and each row already has main category
    # so am taking out all instances where business category could not be found
    lis = set(map((lambda x : str(dfrow['price']) + x),lis))
    return ' '.join(lis)

def get_alias2(cat) :
    try :
        alias = '_' + str(cat_alias_dict[map_category(cat_alias_dict[cat])])
    except : alias = '_other'
    return alias

def categories_to_alias(dfrow) :
    catstring = dfrow['categories']
    if type(catstring) != str : return str(dfrow['price']) + '_other'
    lis = list(set(catstring.split(', ')))
    lis = map(get_alias,lis)
    lis = set(map((lambda x : str(dfrow['price']) + x),lis))
    return ' '.join(lis)

def get_alias(cat) :
    try :
        alias = '_' + str(cat_alias_dict[map_category(cat.lower())])
    except : alias = '_other'
    return alias

## Function to map titles to macro-categories if not in top 150

dict_2A = json.load(open('dict_2A.json','r'))
dict_2B = json.load(open('dict_2B.json','r'))

def map_category(category):
    if category not in top_148:
        if category in dict_2B.keys():
            title = dict_2B[category]
        else:
            title = category
        if title in dict_2A.keys():
            title_2 = dict_2A[title]
            return(title_2)
        else:
            return(title)
    else:
        return(category)

In [13]:
df['alias_cats'] = df.apply(categories_to_alias2, axis=1)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 8431 entries, (25.733, -80.2346) to (25.839, -80.2234)
Data columns (total 16 columns):
Unnamed: 0       8431 non-null int64
alias            8431 non-null object
categories       8431 non-null object
coordinates      8431 non-null object
distance         8431 non-null float64
latitude         8431 non-null float64
longitude        8431 non-null float64
main_category    8431 non-null object
name             8431 non-null object
price            8431 non-null int64
rating           8431 non-null float64
review_count     8431 non-null float64
distance2        8431 non-null float64
lat_index        8431 non-null float64
long_index       8431 non-null float64
alias_cats       8431 non-null object
dtypes: float64(8), int64(2), object(6)
memory usage: 1.1+ MB


In [15]:
df.head()

Unnamed: 0                                   alias  \
lat_index long_index                                                       
25.733    -80.2346         11763                   2601-tower-cafe-miami   
25.734    -80.2397          6765  coconut-grove-dog-walker-coconut-grove   
          -80.2368         13960            desert-sage-soy-candle-miami   
25.735    -80.2559         11833           riconcito-superlatino-4-miami   
          -80.2559         11834            el-rinconcito-el-chele-miami   

                                                             categories  \
lat_index long_index                                                      
25.733    -80.2346    [{'alias': 'cafes', 'title': 'Cafes'}, {'alias...   
25.734    -80.2397    [{'alias': 'pet_sitting', 'title': 'Pet Sittin...   
          -80.2368    [{'alias': 'candlestores', 'title': 'Candle St...   
25.735    -80.2559      [{'alias': 'latin', 'title': 'Latin American'}]   
          -80.2559               [{'alias': 'cuban', 'title': 'Cuban'}]   

                                                            coordinates  \
lat_index long_index                                                      
25.733    -80.2346      {'latitude': 25.732879, 'longitude': -80.23455}   
25.734    -80.2397    {'latitude': 25.7341828197241, 'longitude': -8...   
          -80.2368    {'latitude': 25.7344014, 'longitude': -80.2368...   
25.735    -80.2559    {'latitude': 25.7352829, 'longitude': -80.2558...   
          -80.2559       {'latitude': 25.73527, 'longitude': -80.25592}   

                         distance   latitude  longitude main_category  \
lat_index long_index                                                    
25.733    -80.2346    5968.018614  25.732879 -80.234550   restaurants   
25.734    -80.2397    5821.018862  25.734183 -80.239718          pets   
          -80.2368    5792.654591  25.734401 -80.236815      shopping   
25.735    -80.2559    5987.493746  25.735283 -80.255877   restaurants   
          -80.2559    5990.189762  25.735270 -80.255920   restaurants   

                                          name  price  rating  review_count  \
lat_index long_index                                                          
25.733    -80.2346             2601 Tower Cafe      0     4.5           6.0   
25.734    -80.2397    Coconut Grove Dog Walker      0     5.0           4.0   
          -80.2368      Desert Sage Soy Candle      2     5.0           1.0   
25.735    -80.2559     Riconcito Superlatino 4      0     5.0           1.0   
          -80.2559      El Rinconcito El Chele      1     4.0         116.0   

                        distance2  lat_index  long_index  \
lat_index long_index                                       
25.733    -80.2346    5968.018614     25.733    -80.2346   
25.734    -80.2397    5821.018862     25.734    -80.2397   
          -80.2368    5792.654591     25.734    -80.2368   
25.735    -80.2559    5987.493746     25.735    -80.2559   
          -80.2559    5990.189762     25.735    -80.2559   

                                                             alias_cats  
lat_index long_index                                                     
25.733    -80.2346    0_restaurants 0_cafes 0_sandwiches 0_breakfast...  
25.734    -80.2397                                 0_pet_sitting 0_pets  
          -80.2368                                           2_shopping  
25.735    -80.2559                                        0_restaurants  
          -80.2559                                        1_restaurants

In [16]:

df.to_csv('MiamiBusinessesFinal.csv',sep=';',header=True,index=True,compression='gzip')

In [17]:
df['alias_cats'].map((lambda x : str(len(x))+x)).value_counts()

140_professional                                                     390
80_health                                                            328
140_hotelstravel                                                     296
150_localservices                                                    236
140_homeservices                                                     177
100_shopping                                                         140
80_active                                                            135
190_auto 0_automotive                                                124
102_shopping                                                         113
150_eventservices                                                    109
130_restaurants                                                      105
270_apartments 0_homeservices                                        102
110_education                                                        101
190_financialservices                              

### Census Block Population and Income to X_train Coordinates DataFrame

In [121]:
table = DBF('tl_2014_12_tabblock10.dbf')
type(table)

dbfread.dbf.DBF

In [122]:
df_coords = pd.DataFrame(iter(table))
df_coords.head()

,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE10,GEOID10,NAME10,MTFCC10,UR10,UACE10,UATYP10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10
0,25,011,041502,3034,250110415023034,Block 3034,G5040,R,,,S,0,23531,+42.6023760,-072.7401001
1,25,005,653203,2007,250056532032007,Block 2007,G5040,U,61786,U,S,131659,1089,+41.6192746,-070.9524385
2,25,021,417702,1004,250214177021004,Block 1004,G5040,R,,,S,0,10869,+42.2656702,-070.9883137
3,25,021,423100,4046,250214231004046,Block 4046,G5040,U,09271,U,S,0,529384,+42.2505856,-070.8034428
4,25,019,950200,2006,250199502002006,Block 2006,G5040,U,61003,C,S,455759,72548,+41.2722392,-070.1168998


In [123]:
df_coords = df_coords[['GEOID10','INTPTLAT10','INTPTLON10','UR10','ALAND10','AWATER10']]
# df_coords.to_csv('MiamiCoords.csv',sep=';',header=True,index=True,compression='gzip')

In [124]:
def cut(geoid) :
    index = str(geoid)
    return int(index[0:(len(index)-3)])

def matchincome(geoid) :
    try : return blockgroupincome.loc[geoid,'income']
    except : return 0

def matchpop(geoid) :
    try : return blockgrouppop.loc[geoid,'population']
    except : return 0

In [125]:
table = DBF('tabblock2010_12_pophu.dbf')

populations = pd.DataFrame(iter(table))
populations.head()

,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE,BLOCKID10,PARTFLG,HOUSING10,POP10
0,25,001,011200,4023,250010112004023,N,16,12
1,25,001,011002,2068,250010110022068,N,52,44
2,25,001,011200,4043,250010112004043,N,4,0
3,25,001,011002,2069,250010110022069,N,16,4
4,25,001,011002,3051,250010110023051,N,17,4


In [126]:
populations = populations[['BLOCKID10','POP10']]

In [127]:
populations.columns = ['GEO.id2','population']
populations['blockgroup'] = populations['GEO.id2'].map(cut)
populations['GEO.id2'] = populations['GEO.id2'].map(int)
populations.set_index(['GEO.id2'],inplace=True,drop=False)
populations.head()

,GEO.id2,population,blockgroup
GEO.id2,,,
250010112004023,250010112004023,12,250010112004
250010110022068,250010110022068,44,250010110022
250010112004043,250010112004043,0,250010112004
250010110022069,250010110022069,4,250010110022
250010110023051,250010110023051,4,250010110023


In [128]:
populations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 157508 entries, 250010112004023 to 250277502003013
Data columns (total 3 columns):
GEO.id2       157508 non-null int64
population    157508 non-null int64
blockgroup    157508 non-null int64
dtypes: int64(3)
memory usage: 4.8 MB


In [129]:
blockgroupincome = pd.read_csv('Miami.csv')
blockgroupincome.head()

,Unnamed: 0,GEO.id,GEO.id2,GEO.display-label,HD01_VD01,HD02_VD01,income
0,0,Id,Id2,Geography,Estimate; Total,Margin of Error; Total,Estimate; Per capita income in the past 12 mon...
1,1,1500000US250250001001,250250001001,"Block Group 1, Census Tract 1, Suffolk County,...",2086,482,25693
2,2,1500000US250250001002,250250001002,"Block Group 2, Census Tract 1, Suffolk County,...",918,274,45835
3,3,1500000US250250001003,250250001003,"Block Group 3, Census Tract 1, Suffolk County,...",536,218,41421
4,4,1500000US250250001004,250250001004,"Block Group 4, Census Tract 1, Suffolk County,...",1128,323,36519


In [130]:
blockgroupincome.drop(0, axis=0, inplace=True)
blockgroupincome['GEO.id2'] = blockgroupincome['GEO.id2'].map(int)
blockgroupincome.set_index('GEO.id2', inplace=True)
blockgroupincome.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 646 entries, 250250001001 to 250259901010
Data columns (total 6 columns):
Unnamed: 0           646 non-null int64
GEO.id               646 non-null object
GEO.display-label    646 non-null object
HD01_VD01            646 non-null object
HD02_VD01            646 non-null object
income               645 non-null object
dtypes: int64(1), object(5)
memory usage: 35.3+ KB


In [131]:
populations['income'] = populations['blockgroup'].map(matchincome)
populations['income'].value_counts()

0         148338
-            343
118501       109
104150       103
59502         97
14699         88
10767         83
30422         83
46165         73
132239        71
89477         70
88901         57
13827         57
17785         51
25693         51
23177         46
56396         43
27351         42
7824          42
36434         42
56412         41
7112          40
31415         39
40094         38
40101         38
38735         38
23665         37
26105         37
32355         36
52230         35
28170         35
28951         34
22061         34
31097         34
143313        34
27813         33
42539         32
10021         32
45749         32
30884         31
32917         31
22121         31
64911         31
32804         30
25695         30
54355         30
20922         29
35780         29
38787         28
25881         28
32887         28
23192         28
30354         28
46900         28
39908         27
28009         27
37787         27
28941         27
38263         

In [14]:
blockgrouppop = blockgroupincome

# Population by census block can replace block group population column
# populations['blockgrouppop'] = populations['blockgroup'].map(matchpop)
# populations['blockgrouppop'].value_counts()

C:\Users\Jason\AppData\Local\conda\conda\envs\DSI-6\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


0         168987
17326        787
14746        753
78229        661
18185        639
22581        622
31203        573
3880         531
33868        487
-            419
32150        416
20328        392
18930        301
33053        282
21180        278
19604        269
21896        260
47731        256
21618        239
18767        228
30785        225
16644        203
24263        196
30453        182
18186        172
31979        171
46435        171
12395        171
15586        160
26170        157
39747        154
43373        152
24003        150
29197        150
26288        145
25530        138
12132        134
23799        133
112222       126
20814        122
17091        119
12895        116
43901        114
35941        113
33372        112
35455        111
28170        111
20964        109
25307        108
38521        107
21718        105
36011        104
27142        101
32154        100
15428        100
54499         98
38691         98
18003         97
34027         

In [138]:
populations.head()

,GEO.id2,population,blockgroup,income
GEO.id2,,,,
250010112004023,250010112004023,12,250010112004,0
250010110022068,250010110022068,44,250010110022,0
250010112004043,250010112004043,0,250010112004,0
250010110022069,250010110022069,4,250010110022,0
250010110023051,250010110023051,4,250010110023,0


### X_train Coordinates DataFrame

In [132]:
features = ['distance','urban','latitude','longitude','blockgroup','income',
                'blockgrouppop','population','landarea','waterarea','block']

In [133]:
def blocksdropna(df) :
    df.drop(df[df['income']==0].index, inplace=True)
    mask = df['income'].astype(str)=='-'
    df.drop(df[mask].index, inplace=True)
    df.drop(df[df['population']<20].index, inplace=True)  # min 20 people for block to be counted

In [134]:
# blocks = pd.read_csv('MiamiCoords.csv',sep=';',compression='gzip')
blocks = df_coords
blocks.head()

,GEOID10,INTPTLAT10,INTPTLON10,UR10,ALAND10,AWATER10
0,250110415023034,+42.6023760,-072.7401001,R,0,23531
1,250056532032007,+41.6192746,-070.9524385,U,131659,1089
2,250214177021004,+42.2656702,-070.9883137,R,0,10869
3,250214231004046,+42.2505856,-070.8034428,U,0,529384
4,250199502002006,+41.2722392,-070.1168998,U,455759,72548


In [135]:
# blocks.drop('Unnamed: 0', axis=1, inplace=True)
blocks.columns = ['GEO.id2','latitude','longitude','urban','landarea','waterarea']
blocks['blockgroup'] = blocks['GEO.id2'].map(cut)
blocks['GEO.id2'] = blocks['GEO.id2'].map(int)
blocks = blocks.set_index('GEO.id2')
blocks.sort_index(inplace=True)
blocks.head()

,latitude,longitude,urban,landarea,waterarea,blockgroup
GEO.id2,,,,,,
250010101001000,+42.0828855,-070.1897316,R,0,2684158,250010101001
250010101001001,+42.0807020,-070.1655750,R,0,777331,250010101001
250010101001002,+42.0803222,-070.2045542,R,782775,0,250010101001
250010101001003,+42.0696573,-070.2264801,R,3730412,47750,250010101001
250010101001004,+42.0768983,-070.2156958,R,2051,0,250010101001


In [136]:
blocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 157508 entries, 250010101001000 to 250277614006202
Data columns (total 6 columns):
latitude      157508 non-null object
longitude     157508 non-null object
urban         157508 non-null object
landarea      157508 non-null int64
waterarea     157508 non-null int64
blockgroup    157508 non-null int64
dtypes: int64(3), object(3)
memory usage: 8.4+ MB


In [139]:
blocks = blocks.join(populations[['income','population']], how='inner')
blocks.shape

(157508, 8)

In [140]:
blocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 157508 entries, 250010101001000 to 250277614006202
Data columns (total 8 columns):
latitude      157508 non-null object
longitude     157508 non-null object
urban         157508 non-null object
landarea      157508 non-null int64
waterarea     157508 non-null int64
blockgroup    157508 non-null int64
income        157443 non-null object
population    157508 non-null int64
dtypes: int64(4), object(4)
memory usage: 10.8+ MB


In [141]:
blocksdropna(blocks)
blocks.shape

(5082, 8)

In [142]:
blocks[(blocks['population']!=0) & (blocks['income']==0)] # must drop as don't have y target (income)

,latitude,longitude,urban,landarea,waterarea,blockgroup,income,population
GEO.id2,,,,,,,,


In [143]:
testlat, testlong  # used as center of Miami

(42.349397, -71.09248)

In [144]:
blocks['distance_from_city'] = blocks.apply(
    (lambda row : calcDist(row['latitude'],row['longitude'],testlat, testlong)),         
    axis=1)

In [145]:
blocks = blocks[blocks['distance_from_city'] < (maxradius - 1000)]
blocks.shape

(957, 9)

In [147]:
blocks = blocks.reindex(['distance_from_city','urban','latitude','longitude','blockgroup','income',
                'blockgrouppop','population','landarea','waterarea'], axis=1)
blocks.head()

,distance_from_city,urban,latitude,longitude,blockgroup,income,blockgrouppop,population,landarea,waterarea
GEO.id2,,,,,,,,,,
250250001002006,3407.978390,U,+42.3613320,-071.1305867,250250001002,45835,NaN,61,8749,0
250250001002007,3448.939131,U,+42.3606724,-071.1314840,250250001002,45835,NaN,106,10262,0
250250001003000,3364.654305,U,+42.3603582,-071.1305510,250250001003,41421,NaN,123,13360,0
250250001003001,3306.077241,U,+42.3598663,-071.1300431,250250001003,41421,NaN,124,14996,0
250250001003002,3381.002641,U,+42.3590970,-071.1313771,250250001003,41421,NaN,57,5116,0


In [148]:
blocks[blocks['distance_from_city'] < (maxradius - 1000)][['population']].sum()

population    193816
dtype: int64

In [149]:
def bootstrapBlocks(blocksdf) :  # changed add block rows to per 100 from 200
    df = blocksdf.copy()
    print(df.index)
    for index in blocksdf[blocksdf['population']>100].index : 
        iterpop = int(np.ceil(blocksdf.loc[index,'population'] / 100))
        lis = [pd.Series(data=blocksdf.loc[index].values, name=index+(10**15*i),
                        index=blocksdf.columns) for i in range(1,iterpop)]
#         print(len(lis))
        df = df.append(lis) # more efficient and faster to append all at once
    return df

In [150]:
blocks = bootstrapBlocks(blocks)
blocks.shape

Int64Index([250250001002006, 250250001002007, 250250001003000, 250250001003001,
            250250001003002, 250250001003003, 250250001003010, 250250007031000,
            250250007031007, 250250007031008,
            ...
            250251205003005, 250251206001000, 250251206001001, 250251206001002,
            250251206001004, 250251207001003, 250251207001004, 250251207001007,
            250251207002000, 250251207002001],
           dtype='int64', name='GEO.id2', length=957)


(2400, 10)

In [151]:
blocks.to_csv('MiamiTrainRowCoords.csv',
          sep=';',
          header=True,
          index=True,
          compression='gzip')

## Vectorize Businesses and Assemble X_train

### Construct Category Corpus with Aliases

In [157]:
lat1000m = -115.161605-(-115.172713)

def makecorpus(df, lat, long) :
    distances = calcDistancesV2(df, lat, long)
    distances = pd.DataFrame(data=distances, index=distances.index, columns=['distances'])
    distances = distances.join(df['alias_cats']) #['alias'] = df.loc[distances.index, 'alias']
    print(distances.shape)
    temp = distances.apply(addproximity, axis=1)
    return ' '.join(temp)

def addproximity(dfrow) :
    w = ('near' if (dfrow.loc['distances'] <= 500) else 'far')
    temp = dfrow.loc['alias_cats'].split(' ')
    temp = list(map((lambda x : (x + '_' + w)), temp))
    return ' '.join(temp)

def calcDistancesV2(multiindexdf, lat, long) :
    minlat = lat - lat1000m
    maxlat = lat + lat1000m
    minlong = long - lat1000m
    maxlong = long + lat1000m
#     multiindexdf = multiindexdf.sort_index(level = [0,1])
    idx = pd.IndexSlice
    df = multiindexdf.loc[idx[minlat:maxlat, minlong:maxlong], ['latitude','longitude']]
    distances = pd.Series(index=df.index)
    for index in df.index :
        try :
            distances[index] = calcDist(lat, long, df.loc[index,'latitude'].values[0], 
                                    df.loc[index,'longitude'].values[0])
        except : # catch very rare attribute error to fill in rest of rows
            distances[index] = 10000 # if calcDist gives error, don't include this business
    distances = distances[distances<1000]
    return distances

def make_X_train(coorddf, businessesdf) :
    for index in coorddf.index :
            coorddf.loc[index,'corpus'] = makecorpus(businessesdf, 
                                            coorddf.loc[index,'latitude'], 
                                            coorddf.loc[index,'longitude'])

In [153]:
df = pd.read_csv('MiamiBusinessesFinal.csv',sep=';',compression='gzip',
                 index_col=['lat_index.1','long_index.1'])
df.head()

lat_index  long_index  Unnamed: 0  \
lat_index.1 long_index.1                                      
42.31       -71.1068          42.31    -71.1068        7942   
            -71.1067          42.31    -71.1067        2742   
            -71.1065          42.31    -71.1065       15731   
            -71.1065          42.31    -71.1065         723   
            -71.1054          42.31    -71.1054       13980   

                                                            alias  \
lat_index.1 long_index.1                                            
42.31       -71.1068      mbta-green-street-station-jamaica-plain   
            -71.1067                   curlew-green-jamaica-plain   
            -71.1065                      evergreen-eatery-boston   
            -71.1065                 boston-pilates-jamaica-plain   
            -71.1054       green-street-supermarket-jamaica-plain   

                                                                 categories  \
lat_index.1 long_index.1                                                      
42.31       -71.1068      [{'alias': 'publictransport', 'title': 'Public...   
            -71.1067            [{'alias': 'hair', 'title': 'Hair Salons'}]   
            -71.1065      [{'alias': 'breakfast_brunch', 'title': 'Break...   
            -71.1065      [{'alias': 'healthtrainers', 'title': 'Trainer...   
            -71.1054             [{'alias': 'grocery', 'title': 'Grocery'}]   

                                                                coordinates  \
lat_index.1 long_index.1                                                      
42.31       -71.1068      {'latitude': 42.3104520820164, 'longitude': -7...   
            -71.1067       {'latitude': 42.310483, 'longitude': -71.106699}   
            -71.1065         {'latitude': 42.31047, 'longitude': -71.10652}   
            -71.1065      {'latitude': 42.3104112831871, 'longitude': -7...   
            -71.1054         {'latitude': 42.31015, 'longitude': -71.10543}   

                             distance   latitude  longitude main_category  \
lat_index.1 long_index.1                                                    
42.31       -71.1068      4483.643286  42.310452 -71.106779  hotelstravel   
            -71.1067      4478.610259  42.310483 -71.106699     beautysvc   
            -71.1065      4476.167237  42.310470 -71.106520   restaurants   
            -71.1065      4482.209931  42.310411 -71.106508        active   
            -71.1054      4488.305515  42.310150 -71.105430          food   

                                               name  price  rating  \
lat_index.1 long_index.1                                             
42.31       -71.1068      MBTA Green Street Station      0     4.5   
            -71.1067                   Curlew Green      2     5.0   
            -71.1065               Evergreen Eatery      0     4.5   
            -71.1065                 Boston Pilates      0     5.0   
            -71.1054       Green Street Supermarket      1     3.5   

                          review_count    distance2  \
lat_index.1 long_index.1                              
42.31       -71.1068               8.0  4483.643286   
            -71.1067              16.0  4478.610259   
            -71.1065              34.0  4476.167237   
            -71.1065              11.0  4482.209931   
            -71.1054              11.0  4488.305515   

                                                               alias_cats  
lat_index.1 long_index.1                                                   
42.31       -71.1068                                       0_hotelstravel  
            -71.1067                                   2_beautysvc 2_hair  
            -71.1065      0_restaurants 0_breakfast_brunch 0_tradamerican  
            -71.1065                            0_active 0_healthtrainers  
            -71.1054                                     1_grocery 1_food

In [154]:
df.describe()

,lat_index,long_index,Unnamed: 0,distance,latitude,longitude,price,rating,review_count,distance2
count,13604.000000,13604.000000,13604.000000,13604.000000,13604.000000,13604.000000,13604.000000,13604.000000,13604.000000,13604.000000
mean,42.353293,-71.084485,8461.404734,2640.463843,42.353290,-71.084485,0.752205,3.652088,56.680609,2640.463843
std,0.014890,0.026120,5910.957184,1007.737211,0.014889,0.026120,1.053229,1.199541,167.854290,1007.737211
min,42.310000,-71.147000,0.000000,55.059690,42.309609,-71.147025,0.000000,0.000000,0.000000,55.059690
25%,42.345000,-71.105100,3679.750000,1918.647773,42.344568,-71.105140,0.000000,3.000000,2.000000,1918.647773
50%,42.352000,-71.078900,7403.500000,2748.765261,42.352232,-71.078874,0.000000,4.000000,7.000000,2748.765261
75%,42.363000,-71.061400,12433.250000,3401.419303,42.362730,-71.061379,2.000000,4.500000,35.250000,3401.419303
max,42.390000,-71.037900,24231.000000,4498.146544,42.389657,-71.037936,4.000000,5.000000,6483.000000,4498.146544


In [155]:
blocks = pd.read_csv('MiamiTrainRowCoords.csv',
          sep=';', compression='gzip', index_col='Unnamed: 0')
blocks.head()

,distance_from_city,urban,latitude,longitude,blockgroup,income,blockgrouppop,population,landarea,waterarea
250250001002006,3407.978390,U,42.361332,-71.130587,250250001002,45835,NaN,61,8749,0
250250001002007,3448.939131,U,42.360672,-71.131484,250250001002,45835,NaN,106,10262,0
250250001003000,3364.654305,U,42.360358,-71.130551,250250001003,41421,NaN,123,13360,0
250250001003001,3306.077241,U,42.359866,-71.130043,250250001003,41421,NaN,124,14996,0
250250001003002,3381.002641,U,42.359097,-71.131377,250250001003,41421,NaN,57,5116,0


In [156]:
blocks.describe()

,distance_from_city,latitude,longitude,blockgroup,income,blockgrouppop,population,landarea,waterarea
count,2400.000000,2400.000000,2400.000000,2.400000e+03,2400.000000,0.0,2400.000000,2400.00000,2400.0
mean,2049.898348,42.342952,-71.087410,2.502505e+11,46491.302500,NaN,415.686667,18558.60500,0.0
std,902.402595,0.011812,0.019604,3.462830e+05,37793.580084,NaN,462.138628,22263.30704,0.0
min,104.649419,42.318193,-71.134670,2.502500e+11,4839.000000,NaN,20.000000,260.00000,0.0
25%,1319.097036,42.335056,-71.100599,2.502501e+11,17704.000000,NaN,138.000000,7663.00000,0.0
50%,2106.926862,42.343189,-71.084834,2.502507e+11,31923.000000,NaN,263.500000,12045.00000,0.0
75%,2819.789746,42.351461,-71.072022,2.502508e+11,71444.000000,NaN,485.000000,20263.00000,0.0
max,3499.487720,42.374035,-71.050092,2.502512e+11,165356.000000,NaN,2671.000000,166649.00000,0.0


In [ ]:
make_X_train(blocks, df)

(1002, 2)
(1204, 2)
(1291, 2)
(1377, 2)
(1600, 2)
(1369, 2)
(1508, 2)
(2493, 2)
(2558, 2)
(2521, 2)
(2699, 2)
(2994, 2)
(2770, 2)
(1896, 2)
(1928, 2)
(1907, 2)
(1951, 2)
(2647, 2)
(2641, 2)
(1847, 2)
(1877, 2)
(1920, 2)
(3005, 2)
(2975, 2)
(2983, 2)
(3073, 2)
(1917, 2)
(2774, 2)
(2766, 2)
(1940, 2)
(2746, 2)
(1930, 2)
(1931, 2)
(1570, 2)
(1811, 2)
(1707, 2)
(1815, 2)
(1856, 2)
(1834, 2)
(1864, 2)
(1803, 2)
(1826, 2)
(1808, 2)
(1798, 2)
(1831, 2)
(1829, 2)
(1879, 2)
(1666, 2)
(1767, 2)
(332, 2)
(648, 2)
(445, 2)
(590, 2)
(1275, 2)
(765, 2)
(1446, 2)
(1775, 2)
(1778, 2)
(1517, 2)
(1610, 2)
(1651, 2)
(1657, 2)
(1743, 2)
(1776, 2)
(1828, 2)
(1734, 2)
(1666, 2)
(1726, 2)
(1488, 2)
(447, 2)
(170, 2)
(790, 2)
(1372, 2)
(1568, 2)
(1430, 2)
(1398, 2)
(1341, 2)
(1394, 2)
(1338, 2)
(1229, 2)
(1427, 2)
(1423, 2)
(1840, 2)
(1844, 2)
(4877, 2)
(6475, 2)
(6485, 2)
(6873, 2)
(5147, 2)
(6821, 2)
(4804, 2)
(1619, 2)
(1715, 2)
(2479, 2)
(2324, 2)
(2551, 2)
(2624, 2)
(1947, 2)
(1623, 2)
(7191, 2)
(2600, 2

In [ ]:
blocks.to_csv('CorpusMiamiTrainRowCoords.csv',sep=';',compression='gzip')
blocks.info()

In [90]:
X_features = joblib.load('X_features.joblib')

vect = CountVectorizer(dtype='uint16',min_df=1, vocabulary=X_features,
                       decode_error='ignore', lowercase=True) 

X_sparse = vect.transform(blocks['corpus'])

In [91]:
dfvect = pd.DataFrame(X_sparse.toarray(),dtype='uint16',columns=X_features)
dfvect.head()

,landarea,waterarea,0_restaurants_near,0_restaurants_far,1_restaurants_near,1_restaurants_far,2_restaurants_near,2_restaurants_far,3_restaurants_near,3_restaurants_far,4_restaurants_near,4_restaurants_far,0_shopping_near,0_shopping_far,1_shopping_near,1_shopping_far,2_shopping_near,2_shopping_far,3_shopping_near,3_shopping_far,4_shopping_near,4_shopping_far,0_food_near,0_food_far,1_food_near,1_food_far,2_food_near,2_food_far,3_food_near,3_food_far,4_food_near,4_food_far,0_beautysvc_near,0_beautysvc_far,1_beautysvc_near,1_beautysvc_far,2_beautysvc_near,2_beautysvc_far,3_beautysvc_near,3_beautysvc_far,4_beautysvc_near,4_beautysvc_far,0_homeservices_near,0_homeservices_far,1_homeservices_near,...,2_hair_extensions_far,3_hair_extensions_near,3_hair_extensions_far,4_hair_extensions_near,4_hair_extensions_far,0_lawyers_near,0_lawyers_far,1_lawyers_near,1_lawyers_far,2_lawyers_near,2_lawyers_far,3_lawyers_near,3_lawyers_far,4_lawyers_near,4_lawyers_far,0_french_near,0_french_far,1_french_near,1_french_far,2_french_near,2_french_far,3_french_near,3_french_far,4_french_near,4_french_far,0_selfstorage_near,0_selfstorage_far,1_selfstorage_near,1_selfstorage_far,2_selfstorage_near,2_selfstorage_far,3_selfstorage_near,3_selfstorage_far,4_selfstorage_near,4_selfstorage_far,0_physicaltherapy_near,0_physicaltherapy_far,1_physicaltherapy_near,1_physicaltherapy_far,2_physicaltherapy_near,2_physicaltherapy_far,3_physicaltherapy_near,3_physicaltherapy_far,4_physicaltherapy_near,4_physicaltherapy_far
0,0,0,3,38,6,89,7,112,2,20,0,5,0,16,0,7,0,51,0,24,0,4,1,59,5,113,2,182,0,27,0,0,0,4,0,2,0,20,0,9,0,1,0,217,0,...,1,0,0,0,0,4,13,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,32,31,21,148,57,155,14,23,4,2,1,21,0,9,29,53,3,27,0,3,59,27,93,87,155,85,27,5,0,0,0,5,0,1,4,15,2,10,1,0,188,125,0,...,0,0,0,0,0,10,7,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,31,16,9,125,50,118,7,25,0,6,0,19,0,12,29,52,3,27,0,5,58,11,85,65,147,53,26,4,0,0,0,4,0,2,3,14,1,11,1,0,185,60,0,...,0,0,0,0,0,6,11,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,2,38,8,104,15,101,2,21,0,5,0,10,0,6,1,49,0,20,0,4,3,62,5,124,4,176,0,29,0,0,0,4,0,2,0,20,1,9,1,0,0,222,0,...,1,0,0,0,0,2,15,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,34,16,17,16,49,2,12,0,4,0,1,0,0,1,31,0,3,0,0,0,61,13,93,5,155,0,27,0,0,1,0,0,0,0,2,1,2,1,0,23,191,0,...,0,0,0,0,0,1,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [93]:
joblib.dump(dfvect, 'X_trainMiami.joblib', compress=9)

['X_trainPittsburgh.joblib']

In [94]:
y_trainMiami = blocks['income']

In [95]:
joblib.dump(y_trainMiami, 'y_trainMiami.joblib', compress=9)

['y_trainPittsburgh.joblib']